# Machine Learning Task 1 – Coding Week IIT Guwahati

**Project:** The World Tour of '99 – The Data Detective  
**Objective:** Predict Crowd Energy (0–100) for future venues  
**Approach:** Data cleaning, EDA, feature engineering, model training with hyperparameter tuning  

⚠️ Note:
- Test set is NOT used during training (to avoid data leakage)
- Lead singer's notes are treated as hypotheses, not ground truth

In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

import warnings
warnings.filterwarnings("ignore")


In [ ]:
train_df = pd.read_csv("tour_logs_train.csv")
test_df = pd.read_csv("tour_logs_test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

train_df.head()


In [ ]:
train_df.info()


In [ ]:
train_df.describe(include="all").T


In [ ]:
list(train_df.columns)


In [ ]:
train_df.head(3)


## Feature Leakage Audit

Based on problem constraints, only features available before a show are used.

Dropped features:
- Gig_ID: Identifier, no predictive value
- Crowd_Size: Final attendance known post-show (data leakage)
- Merch_Sales_Post_Show: Recorded after the show (hard leakage)

Remaining features are either known or reasonably estimable before the show.


In [ ]:
leakage_cols = [
    "Gig_ID",
    "Crowd_Size",
    "Merch_Sales_Post_Show"
]

train_df = train_df.drop(columns=leakage_cols)
test_df = test_df.drop(columns=leakage_cols)

train_df.shape, test_df.shape


In [ ]:
train_df["Show_DateTime"] = pd.to_datetime(
    train_df["Show_DateTime"], errors="coerce"
)

test_df["Show_DateTime"] = pd.to_datetime(
    test_df["Show_DateTime"], errors="coerce"
)


In [ ]:
train_df["Show_DateTime"].isna().sum()


In [ ]:
for df in [train_df, test_df]:
    df["Show_Hour"] = df["Show_DateTime"].dt.hour
    df["Show_Month"] = df["Show_DateTime"].dt.month
    df["Is_Weekend"] = df["Day_of_Week"].isin([5, 6]).astype(int)


In [ ]:
train_df = train_df.drop(columns=["Show_DateTime"])
test_df = test_df.drop(columns=["Show_DateTime"])


## Date & Time Feature Engineering

The raw Show_DateTime column was converted into:
- Show_Hour (time of day)
- Show_Month (seasonality)
- Is_Weekend (behavioral effect)

Raw datetime was dropped to avoid high cardinality noise.


In [ ]:
train_df[["Show_Hour", "Show_Month"]].isna().sum()


In [ ]:
hour_median = train_df["Show_Hour"].median()
month_mode = train_df["Show_Month"].mode()[0]

for df in [train_df, test_df]:
    df["Show_Hour"] = df["Show_Hour"].fillna(hour_median)
    df["Show_Month"] = df["Show_Month"].fillna(month_mode)


In [ ]:
train_df[["Show_Hour", "Show_Month"]].isna().sum()


## Handling Missing Date-Time Values

Some shows had invalid or missing datetime entries.
Instead of dropping rows, we imputed:
- Show_Hour using the median hour
- Show_Month using the most frequent month

This preserves dataset size while maintaining realistic temporal distributions.


In [ ]:
train_df["Ticket_Price"].head(10)


In [ ]:
train_df["Ticket_Price"].describe()


In [ ]:
import re

def parse_ticket_price(price):
    if pd.isna(price):
        return np.nan

    price = str(price).strip()

    if price.lower() == "free":
        return 0.0

    # Pound
    if price.startswith("£"):
        return float(price[1:]) * 1.27

    # Euro
    if price.startswith("€"):
        return float(price[1:]) * 1.09

    # Extract first numeric value (handles "$56.79 (VIP: $85.19)")
    numbers = re.findall(r"\d+\.?\d*", price)
    if numbers:
        return float(numbers[0])

    return np.nan


In [ ]:
for df in [train_df, test_df]:
    df["Ticket_Price_USD"] = df["Ticket_Price"].apply(parse_ticket_price)


In [ ]:
train_df[["Ticket_Price", "Ticket_Price_USD"]].head(12)


In [ ]:
train_df = train_df.drop(columns=["Ticket_Price"])
test_df = test_df.drop(columns=["Ticket_Price"])


In [ ]:
train_df["Ticket_Price_USD"].describe()


In [ ]:
median_price = train_df["Ticket_Price_USD"].median()

for df in [train_df, test_df]:
    df["Ticket_Price_USD"] = df["Ticket_Price_USD"].clip(lower=0)
    df["Ticket_Price_USD"] = df["Ticket_Price_USD"].fillna(median_price)


## Ticket Price Normalization

Ticket prices were recorded in mixed currencies (£, €, $) and included special values like "Free".
All prices were converted to USD using provided exchange rates:
- £1 ≈ $1.27
- €1 ≈ $1.09

Free tickets were mapped to $0, and missing/invalid values were imputed using the median price.


In [ ]:
num_cols = ["Volume_Level", "Opener_Rating"]

train_df[num_cols].describe()


In [ ]:
(train_df[num_cols] == 0).sum()


In [ ]:
for col in num_cols:
    train_df.loc[train_df[col] <= 0, col] = np.nan
    test_df.loc[test_df[col] <= 0, col] = np.nan


In [ ]:
for col in num_cols:
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    for df in [train_df, test_df]:
        df[col] = df[col].clip(lower, upper)


In [ ]:
for col in num_cols:
    median_val = train_df[col].median()
    train_df[col] = train_df[col].fillna(median_val)
    test_df[col] = test_df[col].fillna(median_val)


In [ ]:
train_df[num_cols].isna().sum()


## Numeric Feature Cleaning

Zero or negative sensor values were treated as missing due to likely sensor failure.
Outliers were capped using the IQR method to prevent distortion.
Missing values were imputed using median values for robustness.


In [ ]:
cat_cols = ["Venue_ID", "Weather", "Moon_Phase", "Band_Outfit"]

for col in cat_cols:
    print(f"\n{col}")
    print(train_df[col].value_counts(dropna=False))


In [ ]:
for col in cat_cols:
    train_df[col] = train_df[col].fillna("Unknown")
    test_df[col] = test_df[col].fillna("Unknown")


## Categorical Feature Handling

Missing categorical values were grouped into an 'Unknown' category
to ensure robustness against unseen or rare values in the test set.


In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x="Venue_ID", y="Crowd_Energy", data=train_df)
plt.title("Crowd Energy Distribution by Venue")
plt.show()


In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(
    x="Volume_Level",
    y="Crowd_Energy",
    hue="Venue_ID",
    data=train_df
)
plt.title("Volume Level vs Crowd Energy by Venue")
plt.show()


In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(
    x="Ticket_Price_USD",
    y="Crowd_Energy",
    hue="Venue_ID",
    data=train_df
)
plt.title("Ticket Price vs Crowd Energy")
plt.show()


## Exploratory Data Analysis – Key Observations

- Crowd energy varies significantly across venues, confirming venue-specific dynamics.
- Volume level shows diminishing returns in certain venues, supporting noise limit hypotheses.
- Ticket price impacts crowd energy differently across venues, with premium venues showing nonlinear behavior.


In [ ]:
train_df["Crowd_Energy"].describe()


In [ ]:
train_df[train_df["Crowd_Energy"] > 100]


In [ ]:
invalid_rows = train_df[train_df["Crowd_Energy"] > 100].index
len(invalid_rows)


In [ ]:
train_df = train_df.drop(index=invalid_rows).reset_index(drop=True)


In [ ]:
train_df["Crowd_Energy"].describe()


## Target Variable Validation

Crowd_Energy is defined on a 0–100 scale.
A small number of records contained impossible values (~1000), likely due to data entry errors.
These rows were removed to prevent corrupting model training.


In [ ]:
invalid_low = train_df[train_df["Crowd_Energy"] < 0].index
len(invalid_low)


In [ ]:
train_df = train_df.drop(index=invalid_low).reset_index(drop=True)


In [ ]:
train_df["Crowd_Energy"].describe()


Additional validation revealed a small number of records with negative Crowd_Energy values.
Since Crowd_Energy is defined on a 0–100 scale, these rows were removed as corrupted labels.


In [ ]:
y = train_df["Crowd_Energy"]
X = train_df.drop(columns=["Crowd_Energy"])


In [ ]:
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

numeric_features, categorical_features


In [ ]:
X["Price_x_Venue"] = X["Ticket_Price_USD"] * X["Venue_ID"].astype("category").cat.codes
X["Volume_x_Venue"] = X["Volume_Level"] * X["Venue_ID"].astype("category").cat.codes


In [ ]:
test_X = test_df.copy()
test_X["Price_x_Venue"] = test_X["Ticket_Price_USD"] * test_X["Venue_ID"].astype("category").cat.codes
test_X["Volume_x_Venue"] = test_X["Volume_Level"] * test_X["Venue_ID"].astype("category").cat.codes


## Feature Engineering

Venue-specific interaction features were created to capture differing effects
of ticket price and volume across venues, reflecting distinct audience dynamics.


In [ ]:
numeric_features
categorical_features


In [ ]:
numeric_features


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [ ]:
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


In [ ]:
from sklearn.linear_model import Ridge

baseline_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", Ridge())
])


In [ ]:
X = train_df.drop(columns=["Crowd_Energy"])
y = train_df["Crowd_Energy"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_val)
rmse_baseline = np.sqrt(mean_squared_error(y_val, y_pred))
rmse_baseline



## Baseline Model

A Ridge Regression model was used as a baseline due to its robustness and interpretability.
This provides a benchmark for evaluating more complex tuned models.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(random_state=42))
])


In [ ]:
param_grid = {
    "model__n_estimators": [100, 200],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_split": [2, 5],
    "model__min_samples_leaf": [1, 2]
}


In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
grid_search = GridSearchCV(
    rf_pipeline,
    param_grid,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

grid_search.fit(X_train, y_train)


In [ ]:
best_model = grid_search.best_estimator_

y_pred_rf = best_model.predict(X_val)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
rmse_rf



## Hyperparameter Tuning

A Random Forest Regressor was tuned using GridSearchCV with 5-fold cross-validation.
Parameters explored included tree depth, number of estimators, and minimum samples.
The tuned model outperformed the baseline Ridge regression.


In [ ]:
rmse_baseline, rmse_rf


In [ ]:
best_model = grid_search.best_estimator_

X_full = train_df.drop(columns=["Crowd_Energy"])
y_full = train_df["Crowd_Energy"]

best_model.fit(X_full, y_full)


In [ ]:
test_predictions = best_model.predict(test_df)
test_predictions = np.clip(test_predictions, 0, 100)


In [ ]:
submission = pd.DataFrame({
    "Crowd_Energy": test_predictions
})

submission.to_csv("predictions.csv", index=False)


In [ ]:
submission.shape
submission.head()


## Final Model Summary

- Baseline Model: Ridge Regression (RMSE ≈ 18.13)
- Final Model: Random Forest Regressor with hyperparameter tuning
- Validation Strategy: 5-fold cross-validation using GridSearchCV
- Final RMSE: ≈ 16.80

The tuned Random Forest model was selected due to its superior performance
and ability to capture non-linear relationships across venues.


# Bonus: Revenue Optimization for V_Gamma (The Snob Pit)


## Business Assumptions & Profit Model

- Analysis is restricted to V_Gamma.
- Venue capacity is approximately 800 seats.
- Fixed cost per show is $5,000.
- Variable cost per attendee is $8.
- Crowd energy affects attendance and spending behavior.

Attendance model:
Attendance = min(800, 8 × Crowd_Energy)

Profit formula:
Profit = (Ticket_Price × Attendance) − (5000 + 8 × Attendance)


In [ ]:
# Baseline characteristics for a typical V_Gamma show
vgamma_base = train_df[train_df["Venue_ID"] == "V_Gamma"].median(numeric_only=True)
vgamma_base


In [ ]:
vgamma_mode = (
    train_df[train_df["Venue_ID"] == "V_Gamma"]
    [["Weather", "Moon_Phase", "Band_Outfit"]]
    .mode()
    .iloc[0]
)

vgamma_mode


In [ ]:
price_range = np.arange(20, 121, 2)

profits = []
energies = []

for price in price_range:
    temp = vgamma_base.copy()

    # numeric updates
    temp["Ticket_Price_USD"] = price
    temp["Venue_ID"] = "V_Gamma"

    # categorical defaults (REQUIRED)
    temp["Weather"] = vgamma_mode["Weather"]
    temp["Moon_Phase"] = vgamma_mode["Moon_Phase"]
    temp["Band_Outfit"] = vgamma_mode["Band_Outfit"]

    temp_df = pd.DataFrame([temp])

    predicted_energy = best_model.predict(temp_df)[0]
    predicted_energy = np.clip(predicted_energy, 0, 100)

    attendance = min(800, 8 * predicted_energy)

    revenue = price * attendance
    cost = 5000 + 8 * attendance
    profit = revenue - cost

    profits.append(profit)
    energies.append(predicted_energy)


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(price_range, profits)
plt.xlabel("Ticket Price (USD)")
plt.ylabel("Profit (USD)")
plt.title("Profit vs Ticket Price for V_Gamma")
plt.grid(True)
plt.show()


In [ ]:
optimal_idx = np.argmax(profits)
optimal_price = price_range[optimal_idx]
optimal_profit = profits[optimal_idx]

optimal_price, optimal_profit
